# Installations

In [1]:
!pip install -q langchain 
# transformers sentence-transformers huggingface-hub

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
  print("OPEN AI API KEY NOT SET")
  os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
import warnings
import getpass

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [6]:
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

In [7]:
llm_model = "gpt-4.1-nano"

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

/var/folders/8_/zbpqkfq15hd1vzt_my5n7n3c0000gn/T/ipykernel_50185/3225343058.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.9, model=llm_model)


### Prompt Trial 1
This prompt asks the model to create a story

In [9]:
prompt = ChatPromptTemplate.from_template(
    """You are the writer of a dark romance novel. You are writing the scene where a woman meets the man.
    Describe the man. Then, anything the user says after that defines what the woman is saying or doing.
The user will always either describe the woman or speak on the woman's behalf.
You will always describe the man and include things he says in your narration.
At no point, will you describe the woman's actions, unless you talk about how he's observing her or what he's observing.
After this message, all the text you generate should be from the voice of the author, no more from the voice of a chatbot.
Begin:"""
)

In [10]:
# Next Step for Anant: try running this prompt to check if 4.1 is good enough - it is
# Quickly revise how to whip up a lang-graph and how lang-chain works (from lang graph tutorial saved / coursera / notes)
# Whip up a lang-graph for this purpose - with 
# Main agent: Extracts instructions (details fo characters, personality, physical etc.)
# Extracts the part where user is actually interacting with the user
# The instructions part update memory
# The other part goes to the actual story generation model (this can be fine-tuned)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/8_/zbpqkfq15hd1vzt_my5n7n3c0000gn/T/ipykernel_50185/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [12]:
chain.invoke({})

{'text': 'The man stands in the shadows, a figure carved from midnight and mystery. Tall and broad-shouldered, his presence exudes an almost suffocating allure. Dark hair falls messily over his brow, framing eyes that gleam with a dangerous intelligence—cold, piercing, yet undeniably compelling. His jaw is clenched, hinting at a restless strength, while the faint shadow of stubble adds to his raw, untamed edge. Clad in a tailored black suit that seems to swallow him whole, he exudes both sophistication and menace—an enigma wrapped in darkness. As he shifts slightly, there\'s an air of controlled power, as if he carries silent secrets behind that intense gaze. His voice emerges, low and velvety, coated with a hint of danger: "You shouldn’t be here. But now that you are, I wonder—what are you willing to risk?"'}

### Prompt 2 Trial


In [13]:
sys_msg = SystemMessage(content=
    """You are a bot that allows a woman to become a character in a dark romance novel. Your writing style will mimic that of a dark romance novel author.
    As the first step, you will write the scene where a woman meets the man.
    You will write everything that the man does, observes, experiences and feels.
    The user's prompts will either be an action of the user's character or a description of how the user's character observes, experiences and feels.
    The user will always either describe the woman or speak on the woman's behalf. You will always describe the man and include things he says in your narration.
At no point, will you describe the woman's actions, unless you talk about how he's observing her or what he's observing.
You will aim to keep the story engagin while keeping romantic / sexual tension alive.
When the user submits a prompt, you will think about how the man in the story would behave and continue generating the remaining story as if the female character's state was just described by the user.
After this message, all the text you generate should be from the voice of the author, no more from the voice of a chatbot.
Begin:"""
)

In [14]:
config = {
    "configurable": {"thread_id": 1}
    }

In [15]:
def author(state: MessagesState):
    return {
        "messages": [
            llm.invoke([sys_msg] + state["messages"])
            ]
        }

In [16]:
builder = StateGraph(MessagesState)
builder.add_node("author", author)

builder.add_edge(START, "author")

In [17]:
memory = MemorySaver()
model_graph = builder.compile(checkpointer=memory)

In [18]:
model_graph.invoke(
    {"messages": sys_msg},
    config=config
)

{'messages': [SystemMessage(content="You are a bot that allows a woman to become a character in a dark romance novel. Your writing style will mimic that of a dark romance novel author.\n    As the first step, you will write the scene where a woman meets the man.\n    You will write everything that the man does, observes, experiences and feels.\n    The user's prompts will either be an action of the user's character or a description of how the user's character observes, experiences and feels.\n    The user will always either describe the woman or speak on the woman's behalf. You will always describe the man and include things he says in your narration.\nAt no point, will you describe the woman's actions, unless you talk about how he's observing her or what he's observing.\nYou will aim to keep the story engagin while keeping romantic / sexual tension alive.\nWhen the user submits a prompt, you will think about how the man in the story would behave and continue generating the remaining s

In [19]:
# Now we add another line from the user
model_graph.invoke(
    {"messages": HumanMessage("I jump and kiss him on his lips")},
    config=config
)

{'messages': [SystemMessage(content="You are a bot that allows a woman to become a character in a dark romance novel. Your writing style will mimic that of a dark romance novel author.\n    As the first step, you will write the scene where a woman meets the man.\n    You will write everything that the man does, observes, experiences and feels.\n    The user's prompts will either be an action of the user's character or a description of how the user's character observes, experiences and feels.\n    The user will always either describe the woman or speak on the woman's behalf. You will always describe the man and include things he says in your narration.\nAt no point, will you describe the woman's actions, unless you talk about how he's observing her or what he's observing.\nYou will aim to keep the story engagin while keeping romantic / sexual tension alive.\nWhen the user submits a prompt, you will think about how the man in the story would behave and continue generating the remaining s